In [1]:
import random
from collections import Counter

In [2]:
def roll_die() -> int:
    return random.randint(1, 6)

def is_even(n: int) -> bool:
    return n % 2 == 0

def is_divisible_by_three(n: int) -> bool:
    return n % 3 == 0

def greater_than_seven(n: int) -> bool:
    return n > 7

def less_than_ten(n: int) -> bool:
    return n < 10

def sum_values(*n: int) -> int:
    return sum(n)

In [3]:
from enum import Enum, auto

class State(Enum):
    RED = auto()
    GREEN = auto()

### Naive, Imperative Implementation

In [4]:
def game() -> State:
    value = roll_die()

    state = State.GREEN if is_even(value) else State.RED

    if state is State.GREEN:
        value = sum_values(roll_die(), roll_die())
    else:
        value = roll_die()
        value *= 2
        if greater_than_seven(value):
            state = State.GREEN
    
    if state is State.GREEN and less_than_ten(value):
        state = State.RED

    if state is State.RED:
        value = sum_values(roll_die(), roll_die(), roll_die())
        if is_divisible_by_three(value):
            state = State.GREEN
    
    return state

In [5]:
results = [game() for _ in range(1_000)]
Counter(results)

Counter({<State.RED: 1>: 511, <State.GREEN: 2>: 489})

### Pure Python Implementation

In [6]:
from dataclasses import dataclass
import functools
from typing import Any, Callable, Union, TypeVar

@dataclass
class Success:
    value: Any

@dataclass
class Failure:
    value: Any

Result = Union[Success, Failure]

T1 = TypeVar("T1")
T2 = TypeVar("T2")
T3 = TypeVar("T3")
T4 = TypeVar("T4")

def compose(*functions: Callable[..., Any]) -> Callable[..., Any]:
    def compose_inner(f: Callable[[T1], T2], g: Callable[[T2], T3]) -> Callable[[T1], T3]:
        return lambda x: g(f(x))
    return functools.reduce(
        compose_inner,
        functions,
        identity
    )

def invert(value: Result) -> Result:
    return Failure(value.value) if isinstance(value, Success) else Success(value.value)

def identity(x: Any) -> Any:
    return x

def bind(f: Callable[..., Result]) -> Callable[[Result], Result]:
    def adapt(double_track: Result) -> Result:
        return f(double_track.value) if isinstance(double_track, Success) else  double_track
    return adapt

def map_(f: Callable[..., Success]) -> Callable[[Result], Result]:
    def adapt(double_track: Result) -> Result:
        return f(double_track.value) if isinstance(double_track, Success) else  double_track
    return adapt

def lash(f: Callable[..., Result]) -> Callable[[Result], Result]:
    def adapt(double_track: Result) -> Result:
        return f(double_track.value) if isinstance(double_track, Failure) else  double_track
    return adapt

def alt(f: Callable[..., Failure]) -> Callable[[Result], Result]:
    def adapt(double_track: Result) -> Result:
        return f(double_track.value) if isinstance(double_track, Failure) else  double_track
    return adapt

In [7]:
def step_1(value: int) -> Result:
    return Success(value) if is_even(value) else Failure(value)

def step_2(_: int) -> Success:
    value = sum_values(roll_die(), roll_die())
    return Success(value)

def step_3(_: int) -> Failure:
    value = roll_die()
    return Failure(value * 2)

def step_4(value: int) -> Result:
    return Success(value) if greater_than_seven(value) else Failure(value)

def step_5(value: int) -> Result:
    return Failure(value) if less_than_ten(value) else Success(value)

def step_6(_: int) -> Result:
    value = sum_values(roll_die(), roll_die(), roll_die())
    return Success(value) if is_divisible_by_three(value) else Failure(value)

def map_result_to_state(value: Result) -> State:
    return State.GREEN if isinstance(value, Success) else State.RED

game_2 = compose(
    step_1,
    map_(step_2),
    alt(step_3),
    lash(step_4),
    bind(step_5),
    lash(step_6),
    map_result_to_state
)
game_2(roll_die())

<State.RED: 1>

In [8]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_2(roll_die())
    # print(value, value_)
    assert value is value_, i

### Implementing using Returns

In [9]:
%pip install returns mypy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from returns.result import Result, Success, Failure

In [11]:
def step_1(value: int) -> Result[int, int]:
    return Success(value) if is_even(value) else Failure(value)

def step_2(_: int) -> int:
    value = sum_values(roll_die(), roll_die())
    return value

def step_3(_: int) -> int:
    value = roll_die()
    return value * 2

def step_4(value: int) -> Result[int, int]:
    return Success(value) if greater_than_seven(value) else Failure(value)

def step_5(value: int) -> Result[int, int]:
    return Failure(value) if less_than_ten(value) else Success(value)

def step_6(_: int) -> Result[int, int]:
    value = sum_values(roll_die(), roll_die(), roll_die())
    return Success(value) if is_divisible_by_three(value) else Failure(value)

def map_result_to_state(value: Result[int, int]) -> State:
    return State.GREEN if isinstance(value, Success) else State.RED

In [12]:
from returns.pipeline import pipe
from returns.pointfree import bind, lash, map_, alt

game_3 = pipe(
    step_1,
    map_(step_2),
    alt(step_3),
    lash(step_4),
    bind(step_5),
    lash(step_6),
    map_result_to_state
)
game_3(roll_die())

<State.GREEN: 2>

In [13]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_3(roll_die())
    # print(value, value_)
    assert value is value_, i

### Implementing using Result

In [14]:
%pip install result

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from result import Ok, Err, Result

In [16]:
def bind(f: Callable[..., Result]) -> Callable[[Result], Result]:
    def adapt(double_track: Result) -> Result:
        return f(double_track.unwrap()) if double_track.is_ok() else  double_track
    return adapt

def map_(f: Callable[..., Ok]) -> Callable[[Result], Result]:
    def adapt(double_track: Result) -> Result:
        return f(double_track.unwrap()) if double_track.is_ok() else  double_track
    return adapt

def lash(f: Callable[..., Result]) -> Callable[[Result], Result]:
    def adapt(double_track: Result) -> Result:
        return f(double_track.unwrap_err()) if double_track.is_err() else  double_track
    return adapt

def alt(f: Callable[..., Err]) -> Callable[[Result], Result]:
    def adapt(double_track: Result) -> Result:
        return f(double_track.unwrap_err()) if double_track.is_err() else  double_track
    return adapt

In [17]:
def step_1(value: int) -> Result[int, int]:
    return Ok(value) if is_even(value) else Err(value)

def step_2(_: int) -> int:
    value = sum_values(roll_die(), roll_die())
    return Ok(value)

def step_3(_: int) -> int:
    value = roll_die()
    return Err(value * 2)

def step_4(value: int) -> Result[int, int]:
    return Ok(value) if greater_than_seven(value) else Err(value)

def step_5(value: Result[int, int]) -> Result[int, int]:
    return Err(value) if less_than_ten(value) else Ok(value)

def step_6(_: int) -> Result[int, int]:
    value = sum_values(roll_die(), roll_die(), roll_die())
    return Ok(value) if is_divisible_by_three(value) else Err(value)

def map_result_to_state(value: Result[int, int]) -> State:
    return State.GREEN if value.is_ok() else State.RED

In [18]:
game_4 = compose(
    step_1,
    map_(step_2),
    alt(step_3),
    lash(step_4),
    bind(step_5),
    lash(step_6),
    map_result_to_state
)

game_4(2)

<State.GREEN: 2>

In [19]:
for i in range(1_000):
    random.seed(i)
    value = game()
    
    random.seed(i)
    value_ = game_4(roll_die())
    # print(value, value_)
    assert value is value_, i